This example aims to have two target orbits, and one observer orbit. The observer is tasked with monitoring the two targets to minimize the trace covariance of the state over the simulation horizon. We use a PPO agent that is rewarded for minimizing the trace covariance of the target states to the best of its ability.

In [1]:
%matplotlib widget
import sys
sys.path.append("../src")

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset


from SensorTasking import SSA_Problem, Greedy_SSA_Problem, run_experiment
from plotting import visualize_info_vs_phase, render, plot_orbits, _visualize_info_other_observers_fixed

## for latin hypercube generation
from scipy.stats.qmc import LatinHypercube

In [2]:
# L2 Halo orbit, period = 3.225 TU  (Target orbit)
x0_1 = np.array([
        1.1540242813087864,
        0.0,
        -0.1384196144071876,
        4.06530060663289e-15,
        -0.21493019200956867,
        8.48098638414804e-15
    ])

# L1 lyapunov, period = 3.225 TU (target orbit)
x0_2 = np.array([
        0.8027692908754149,
        0.0,
        0.0,
        -1.1309830924549648e-14,
        0.33765564334938736,
        0.0
    ])


# 1:1 L2 Lyapunov, period = 6.45 TU (observer orbit)
x0_3 = np.array([
        0.9982702689023665,
        0.0,
        0.0,
        -2.5322340091977996e-14,
        1.5325475708886613,
        0.0
    ])

# 1:1 L1 Lyapunov, period = 6.45 TU (observer orbit)
x0_4 = np.array([
            0.65457084231188,
        0.0,
        0.0,
        3.887957091335523e-13,
        0.7413347560791179,
        0.0
])


# 2:1 Resonant, period = 6.45 TU
x0_5 = np.array([
        0.9519486347314083,
        0.0,
        0.0,
        0.0,
        -0.952445273435512,
        0.0
    ])

# 3:1 Resonant, period = 6.45 TY
x0_6 = np.array([
        0.13603399956670137,
        0.0,
        0.0,
        1.9130717669166003e-12,
        3.202418276067991,
        0.0
    ])



In [ ]:
opt = "max"
Y = np.array([x0_1, x0_2])
Y_periods = np.array([3.225, 3.225])

X = np.array([x0_4, x0_5])
X_periods = np.array([6.45, 6.45])

prob = SSA_Problem(targets=Y,
                   target_periods=Y_periods,
                   agents=X,
                   agent_periods=X_periods,
                   opt=opt)
plot_orbits(prob, fig=10, projection="xz")
plt.tight_layout()

In [ ]:
plt.figure(1)
# phases = np.linspace(0.385, 0.3865, 40)
phases = np.linspace(0., 1.0, 40)
visualize_info_vs_phase(prob, phases, observer=1, fig=1)
plt.rcParams.update({'font.size': 16})
plt.tight_layout()
plt.show()

show_inset = True

if show_inset:
    plt.axvline(x=0.0134, color='red', linestyle='--', linewidth = 0.5)

    ax = plt.gca()

    # Create inset axes
    ax_inset = inset_axes(ax, width='30%', height='30%', loc='upper center')
    ax.set_ylim(top=7.4e10)

    x = np.linspace(0.01, 0.018, 40)
    y = np.zeros_like(x)

    d = np.zeros(2)
    observer = 1
    for i, phase in enumerate(x):
        d[observer-1] = phase
        _, y[i] = prob.get_control_obj(d)

    # Zoomed in region
    zoom_x = [0.01, 0.018]
    zoom_y = [7.1e10, 7.16e10]

    # Draw the zoomed-in data on the inset
    ax_inset.plot(x, y, color='blue')
    ax_inset.set_xlim(zoom_x)
    ax_inset.set_ylim(zoom_y)
    ax_inset.set_yscale('log')
    ax_inset.yaxis.tick_right()
    plt.axvline(x=0.0134, color='red', linestyle='--', linewidth = 0.5)

    # # Optionally, add a rectangle to highlight the zoomed area on the main plot
    mark_inset(ax, ax_inset, loc1=2, loc2=3, fc='none', ec='black', linewidth = 0.7)



In [ ]:
plt.figure(2)
# phases = np.linspace(0.946, 0.951, 40)
phases = np.linspace(0., 1.0, 40)
visualize_info_vs_phase(prob, phases, observer=2, fig=2)
plt.tight_layout()
plt.rcParams.update({'font.size': 16})

show_inset = True

if show_inset:

    plt.axvline(x=0.0023, color='red', linestyle='--', linewidth = 0.5)

    ax = plt.gca()


    pos = ax.get_position()
    # Adjust the y-coordinate to move it a couple of pixels down
    # The y-coordinate is in normalized figure coordinates (0 to 1)
    y_offset = 0.2  # Adjust this value as needed
    scale = 0.3
    new_position = [pos.x0, pos.y1 - y_offset, scale*pos.width, scale*pos.height]

    # Create inset axes
    ax_inset = ax.inset_axes(bounds=new_position)

    x = np.linspace(0.0, 0.004, 40)
    y = np.zeros_like(x)

    d = np.zeros(2)
    observer = 2
    for i, phase in enumerate(x):
        d[observer-1] = phase
        _, y[i] = prob.get_control_obj(d)

    # Zoomed in region
    zoom_x = [0.00, 0.004]
    zoom_y = [6.29e10, 6.315e10]

    # Draw the zoomed-in data on the inset
    ax_inset.plot(x, y, color='blue')
    ax_inset.set_xlim(zoom_x)
    ax_inset.set_ylim(zoom_y)
    ax_inset.set_yscale('log')
    ax_inset.yaxis.tick_right()
    ax_inset.axvline(x=0.0023, color='red', linestyle='--', linewidth = 0.5)

    # # Optionally, add a rectangle to highlight the zoomed area on the main plot
    mark_inset(ax, ax_inset, loc1=2, loc2=3, fc='none', ec='black', linewidth = 0.7)


In [ ]:
_visualize_info_other_observers_fixed(prob=prob, observer = 1, fig_num = 7)

In [4]:
# latin hypercube sampling
engine = LatinHypercube(d=2, seed=0)
sample = engine.random(n=10)


In [ ]:
phases, control, objective = run_experiment(obj=opt, method="greedy", targets=Y, target_periods=Y_periods, agents=X, agent_periods=X_periods, init_phase_guess=sample)

In [ ]:
phs, ctrl, obj = run_experiment(obj=opt, method="exhaustive", targets=Y, target_periods=Y_periods, agents=X, agent_periods=X_periods, init_phase_guess=sample)

In [ ]:
phases, control, objective = run_experiment(obj=opt, method="ga", targets=Y, target_periods=Y_periods, agents=X, agent_periods=X_periods, init_phase_guess=sample)

In [ ]:
sample